### Cài đặt môi trường

In [1]:
%%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1" # [NEW] Extra 30% context lengths!
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install or uv pip install
    !pip install unsloth vllm
else:
    pass # For Colab / Kaggle, we need extra instructions hidden below \/

In [2]:
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2

### Tải Model Vietnamese Legal Llama3.2-3B

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1536 
lora_rank = 32 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "mikeethanh/vietnamese-legal-llama3.2-3b-merged-sft-v1",
    max_seq_length = max_seq_length,
    load_in_4bit = False, 
    fast_inference = True, 
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.85, 
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-11-30 07:09:27.838904: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764486568.416462      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764486568.622157      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


INFO 11-30 07:10:02 [__init__.py:244] Automatically detected platform cuda.
ERROR 11-30 07:10:04 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 11-30 07:10:21 [vllm_utils.py:702] Unsloth: Patching vLLM v1 graph capture
INFO 11-30 07:10:21 [vllm_utils.py:732] Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.56.2. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading mikeethanh/vietnamese-legal-llama3.2-3b-merged-sft-v1 with actual

`torch_dtype` is deprecated! Use `dtype` instead!


INFO 11-30 07:10:38 [config.py:1472] Using max model len 1536
WARNING 11-30 07:10:38 [arg_utils.py:1735] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 11-30 07:10:40 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 11-30 07:10:40 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.2) with config: model='mikeethanh/vietnamese-legal-llama3.2-3b-merged-sft-v1', speculative_config=None, tokenizer='mikeethanh/vietnamese-legal-llama3.2-3b-merged-sft-v1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1536, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/943 [00:00<?, ?B/s]

Unsloth 2025.11.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


### Chuẩn bị dữ liệu luật giao thông
<a name="Data"></a>



In [4]:
import json
from datasets import Dataset

# Load Vietnamese legal traffic law dataset
data_file = "/kaggle/input/synthetic-traffic-legal/synthetic_legal_qa_data.jsonl"

with open(data_file, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

print(f"✅ Loaded {len(data)} Vietnamese legal Q&A pairs from {data_file}")
print("\n📋 Sample data:")
print(f"❓ Question: {data[0]['question']}")
print(f"💡 Answer: {data[0]['answer'][:150]}...")

# Show data statistics
avg_q_len = sum(len(item['question'].split()) for item in data) / len(data)
avg_a_len = sum(len(item['answer'].split()) for item in data) / len(data)
print(f"\n📊 Dataset statistics:")
print(f"   Average question length: {avg_q_len:.1f} words")
print(f"   Average answer length: {avg_a_len:.1f} words")

✅ Loaded 806 Vietnamese legal Q&A pairs from /kaggle/input/synthetic-traffic-legal/synthetic_legal_qa_data.jsonl

📋 Sample data:
❓ Question: Đường phố và đường bộ có gì khác nhau?
💡 Answer: Theo văn bản pháp luật đã cung cấp, "đường bộ" là khái niệm chung bao gồm tất cả các loại đường, cầu, hầm phục vụ cho phương tiện giao thông, trong kh...

📊 Dataset statistics:
   Average question length: 14.9 words
   Average answer length: 167.6 words


### Định dạng dữ liệu cho GRPO

In [5]:
# Define reasoning markers for structured legal responses
reasoning_start = "<start_working_out>"
reasoning_end   = "<end_working_out>"
solution_start = "<SOLUTION>"
solution_end = "</SOLUTION>"

# Updated system prompt for Vietnamese legal reasoning
system_prompt = f"""Bạn là một trợ lý AI chuyên về luật giao thông Việt Nam. Khi trả lời câu hỏi, hãy:
1. Suy nghĩ và phân tích câu hỏi trong phần {reasoning_start} {reasoning_end}
2. Đưa ra câu trả lời chính xác trong phần {solution_start}{solution_end}

Câu trả lời cần dựa trên quy định pháp luật hiện hành và phải rõ ràng, dễ hiểu."""

print(system_prompt)

Bạn là một trợ lý AI chuyên về luật giao thông Việt Nam. Khi trả lời câu hỏi, hãy:
1. Suy nghĩ và phân tích câu hỏi trong phần <start_working_out> <end_working_out>
2. Đưa ra câu trả lời chính xác trong phần <SOLUTION></SOLUTION>

Câu trả lời cần dựa trên quy định pháp luật hiện hành và phải rõ ràng, dễ hiểu.


In [ ]:

def extract_structured_answer(answer_text):
    """Convert regular answer to structured format with reasoning"""
    return answer_text.strip()

def format_legal_qa_reasoning(examples):
    prompts = []
    answers = []
    
    for question, answer in zip(examples["question"], examples["answer"]):
        prompt = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ]
        prompts.append(prompt)
        
        structured_answer = extract_structured_answer(answer)
        answers.append(structured_answer)
    
    return {
        "prompt": prompts,
        "answer": answers
    }

dataset = Dataset.from_list(data)
dataset = dataset.map(format_legal_qa_reasoning, batched=True, remove_columns=dataset.column_names)
print(f"Formatted {len(dataset)} examples for GRPO training")
print("\nExample formatted data:")
print("Prompt:", dataset[0]["prompt"])
print("Answer:", dataset[0]["answer"][:200] + "...")

Map:   0%|          | 0/806 [00:00<?, ? examples/s]

Formatted 806 examples for GRPO training

Example formatted data:
Prompt: [{'content': 'Bạn là một trợ lý AI chuyên về luật giao thông Việt Nam. Khi trả lời câu hỏi, hãy:\n1. Suy nghĩ và phân tích câu hỏi trong phần <start_working_out> <end_working_out>\n2. Đưa ra câu trả lời chính xác trong phần <SOLUTION></SOLUTION>\n\nCâu trả lời cần dựa trên quy định pháp luật hiện hành và phải rõ ràng, dễ hiểu.', 'role': 'system'}, {'content': 'Đường phố và đường bộ có gì khác nhau?', 'role': 'user'}]
Answer: Theo văn bản pháp luật đã cung cấp, "đường bộ" là khái niệm chung bao gồm tất cả các loại đường, cầu, hầm phục vụ cho phương tiện giao thông, trong khi "đường phố" là một loại cụ thể nằm trong đô thị,...


In [7]:
# Regex pattern to match the structured reasoning format
import re

match_format = re.compile(
    rf"^[\s]{{0,}}"
    rf"{re.escape(reasoning_start)}.+?{re.escape(reasoning_end)}.*?"
    rf"{re.escape(solution_start)}(.+?){re.escape(solution_end)}"
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

# Test the regex pattern
test_response = f"{reasoning_start}Để trả lời câu hỏi này, tôi cần xem xét các quy định về luật giao thông...{reasoning_end}{solution_start}Theo Luật Giao thông đường bộ, mức phạt cho vi phạm này là 1-3 triệu đồng.{solution_end}"

match_result = match_format.search(test_response)
if match_result:
    print("✓ Regex pattern works correctly")
    print(f"Extracted solution: {match_result.group(1)}")
else:
    print("✗ Regex pattern needs adjustment")

✓ Regex pattern works correctly
Extracted solution: Theo Luật Giao thông đường bộ, mức phạt cho vi phạm này là 1-3 triệu đồng.


### Reward Functions cho Vietnamese Legal QA

In [ ]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        score += 0.5 if response.count(reasoning_start) == 1 else -1.0
        score += 0.5 if response.count(reasoning_end)   == 1 else -1.0
        score += 0.5 if response.count(solution_start)  == 1 else -1.0
        score += 0.5 if response.count(solution_end)    == 1 else -1.0
        scores.append(score)
    return scores

### Kiểm tra độ dài prompt tối đa

In [ ]:
max_prompt_length = max(dataset.map(
    lambda x: {"tokens" : tokenizer.apply_chat_template(x["prompt"], add_generation_prompt = True, tokenize = True)},
    batched = True,
).map(lambda x: {"length" : len(x["tokens"])})["length"])

print(f"Maximum prompt length: {max_prompt_length} tokens")
max_prompt_length = min(max_prompt_length + 10, 400)  
print(f"Using max_prompt_length: {max_prompt_length} tokens")

Map:   0%|          | 0/806 [00:00<?, ? examples/s]

Map:   0%|          | 0/806 [00:00<?, ? examples/s]

Maximum prompt length: 126 tokens
Using max_prompt_length: 136 tokens


### Debug và kiểm tra reward functions

In [ ]:
def test_reward_functions_comprehensive():
    """Kiểm tra toàn diện các reward functions với nhiều test cases khác nhau"""
    
    test_cases = [
        {
            "name": "Perfect Format Response",
            "response": f"{reasoning_start}Để trả lời câu hỏi này, tôi cần xem xét quy định về tốc độ trong khu vực đông dân cư theo Luật Giao thông đường bộ. Căn cứ vào Nghị định 100/2019/NĐ-CP, tốc độ tối đa được quy định rõ ràng.{reasoning_end}{solution_start}Theo quy định tại Điều 11 Luật Giao thông đường bộ và Nghị định 100/2019/NĐ-CP, tốc độ tối đa cho phép trong khu vực đông dân cư là 50 km/h đối với ô tô con và 40 km/h đối với xe tải.{solution_end}",
            "expected_high": True
        },
        {
            "name": "No Format - Good Content",
            "response": "Theo quy định của pháp luật Việt Nam, tốc độ tối đa trong khu vực đông dân cư là 50 km/h cho xe con và 40 km/h cho xe tải. Điều này được quy định trong Luật Giao thông đường bộ.",
            "expected_high": False
        },
        {
            "name": "Partial Format - Missing Reasoning",
            "response": f"{solution_start}Theo Nghị định 100/2019/NĐ-CP, tốc độ tối đa trong khu vực đông dân cư là 50 km/h đối với xe con, 40 km/h đối với xe tải.{solution_end}",
            "expected_high": False
        },
        {
            "name": "Partial Format - Missing Solution",
            "response": f"{reasoning_start}Để trả lời câu hỏi về tốc độ, tôi cần tham khảo Luật Giao thông đường bộ và các nghị định liên quan.{reasoning_end}",
            "expected_high": False
        },
        {
            "name": "Wrong Format Order",
            "response": f"{solution_start}50 km/h cho xe con{solution_end}{reasoning_start}Theo luật giao thông{reasoning_end}",
            "expected_high": False
        },
        {
            "name": "Duplicate Tags",
            "response": f"{reasoning_start}First reasoning{reasoning_end}{reasoning_start}Second reasoning{reasoning_end}{solution_start}Answer{solution_end}",
            "expected_high": False
        },
        {
            "name": "Empty Response",
            "response": "",
            "expected_high": False
        },
        {
            "name": "Incomplete Tags",
            "response": f"{reasoning_start}Reasoning without end tag {solution_start}Solution{solution_end}",
            "expected_high": False
        }
    ]
    
    print("🔍 KIỂM TRA CHI TIẾT 2 REWARD FUNCTIONS")
    print("=" * 80)
    print("🎯 Focus: Format Exactly vs Format Approximately")
    print("-" * 80)
    
    # Chỉ có 2 reward functions
    reward_functions = [
        ("Format Exactly", match_format_exactly),
        ("Format Approximately", match_format_approximately),
    ]
    
    for test_case in test_cases:
        print(f"\n📝 Test Case: {test_case['name']}")
        print(f"Expected Quality: {'HIGH' if test_case['expected_high'] else 'LOW'}")
        print(f"Response: {test_case['response'][:80]}{'...' if len(test_case['response']) > 80 else ''}")
        print("-" * 40)
        
        # Prepare test data
        test_completion = [[{"content": test_case['response']}]]
        
        total_score = 0
        for func_name, func in reward_functions:
            try:
                score = func(test_completion)[0]
                total_score += score
                print(f"  {func_name}: {score:.2f}")
            except Exception as e:
                print(f"  {func_name}: ERROR - {str(e)[:50]}...")
        
        print(f"  🎯 TOTAL SCORE: {total_score:.2f}")
        
        # Validate expectations - điều chỉnh threshold cho 2 functions
        expected_threshold = 2.0  # Perfect case: 3.0 + 2.0 = 5.0
        if test_case['expected_high'] and total_score < expected_threshold:
            print("  ⚠️  WARNING: Expected high score but got low!")
        elif not test_case['expected_high'] and total_score >= expected_threshold:
            print("  ⚠️  WARNING: Expected low score but got high!")
        else:
            print("  ✅ Score aligns with expectations")
    
    print("\n" + "=" * 80)
    print("🎉 2-Function reward testing completed!")
    
    print("\n🧪 DETAILED FORMAT TESTING")
    print("=" * 60)
    
    test_formats = [
        ("Perfect Format", f"{reasoning_start}Test reasoning{reasoning_end}{solution_start}Test solution{solution_end}"),
        ("No Format", "Just a regular answer without any tags"),
        ("Only Reasoning", f"{reasoning_start}Only reasoning here{reasoning_end}"),
        ("Only Solution", f"{solution_start}Only solution here{solution_end}"),
        ("Duplicate Reasoning", f"{reasoning_start}First{reasoning_end}{reasoning_start}Second{reasoning_end}{solution_start}Solution{solution_end}"),
    ]
    
    for format_name, test_response in test_formats:
        print(f"\n📋 Testing {format_name}:")
        print(f"   Response: {test_response[:60]}...")
        test_completion = [[{"content": test_response}]]
        
        exactly_score = match_format_exactly(test_completion)[0]
        approx_score = match_format_approximately(test_completion)[0]
        
        print(f"   Format Exactly: {exactly_score:.1f}")
        print(f"   Format Approximately: {approx_score:.1f}")
        print(f"   Combined Score: {exactly_score + approx_score:.1f}")
    
    print("\n" + "=" * 60)
    print("✅ Format testing analysis completed!")
    return True

# Run comprehensive testing
test_reward_functions_comprehensive()

🔍 KIỂM TRA CHI TIẾT 2 REWARD FUNCTIONS
🎯 Focus: Format Exactly vs Format Approximately
--------------------------------------------------------------------------------

📝 Test Case: Perfect Format Response
Expected Quality: HIGH
Response: <start_working_out>Để trả lời câu hỏi này, tôi cần xem xét quy định về tốc độ tr...
----------------------------------------
  Format Exactly: 3.00
  Format Approximately: 2.00
  🎯 TOTAL SCORE: 5.00
  ✅ Score aligns with expectations

📝 Test Case: No Format - Good Content
Expected Quality: LOW
Response: Theo quy định của pháp luật Việt Nam, tốc độ tối đa trong khu vực đông dân cư là...
----------------------------------------
  Format Exactly: 0.00
  Format Approximately: -4.00
  🎯 TOTAL SCORE: -4.00
  ✅ Score aligns with expectations

📝 Test Case: Partial Format - Missing Reasoning
Expected Quality: LOW
Response: <SOLUTION>Theo Nghị định 100/2019/NĐ-CP, tốc độ tối đa trong khu vực đông dân cư...
----------------------------------------
  Format Exact

True

### Cấu hình GRPO Training tối ưu cho T4 15GB

In [ ]:
from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    learning_rate = 3e-6,  
    weight_decay = 0.05,   
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",  
    logging_steps = 1,
    per_device_train_batch_size = 1,  
    gradient_accumulation_steps = 8,  
    num_generations = 4,   
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    num_train_epochs = 1,
    save_steps = 100,     
    max_grad_norm = 1.0,
    report_to = "none",
    output_dir = "outputs",
    seed = 42
)

print("Training configuration optimized for T4 15GB VRAM")
print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"Max sequence length: {max_seq_length}")
print(f"Max prompt length: {training_args.max_prompt_length}")
print(f"Max completion length: {training_args.max_completion_length}")

Training configuration optimized for T4 15GB VRAM
Effective batch size: 8
Max sequence length: 1536
Max prompt length: 136
Max completion length: 1400


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,              
        match_format_approximately,             
    ],
    args = training_args,
    train_dataset = dataset,
)

print("🚀 Vietnamese Legal GRPO Trainer initialized!")
print("📊 Training metrics to watch:")
print("  - reward: Should increase as model learns Vietnamese legal reasoning")
print("  - reward_std: Standard deviation of rewards")
print("  - completion_length: Average Vietnamese response length")

print("  - kl: KL divergence (should stay reasonable < 1.0)")



🚀 Vietnamese Legal GRPO Trainer initialized!
📊 Training metrics to watch:
  - reward: Should increase as model learns Vietnamese legal reasoning
  - reward_std: Standard deviation of rewards
  - completion_length: Average Vietnamese response length
  - kl: KL divergence (should stay reasonable < 1.0)
🎯 Goal: Train model to follow reasoning format consistently (content already good from SFT)
📝 Primary focus: <start_working_out>reasoning<end_working_out><SOLUTION>answer</SOLUTION>
⚡ Format structure has 3.0x weight - highest priority for reasoning enforcement


In [ ]:
# Start training
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 806 | Num Epochs = 1 | Total steps = 403
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 48,627,712 of 3,261,377,536 (1.49% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / match_format_exactly / mean,rewards / match_format_exactly / std,rewards / match_format_approximately / mean,rewards / match_format_approximately / std
1,0.000000,-4.000000,0.000000,688.875000,182.000000,1400.000000,0.375000,262.200012,182.000000,400.000000,0.000008,0.000000,0.000000,-4.000000,0.000000
2,0.000000,-3.437500,1.125000,900.500000,305.000000,1400.000000,0.375000,600.799988,305.000000,1229.000000,0.000008,0.000000,0.000000,-3.437500,1.590990
3,0.000000,-4.000000,0.000000,265.500000,124.000000,564.000000,0.000000,265.500000,124.000000,564.000000,0.000012,0.000000,0.000000,-4.000000,0.000000
4,0.000000,-4.000000,0.000000,603.625000,109.000000,1400.000000,0.250000,338.166687,109.000000,736.000000,0.000008,0.000000,0.000000,-4.000000,0.000000
5,0.000000,-4.000000,0.000000,882.000000,17.000000,1400.000000,0.375000,571.200012,17.000000,1210.000000,0.000007,0.000000,0.000000,-4.000000,0.000000
6,0.000000,-4.000000,0.000000,783.875000,243.000000,1400.000000,0.375000,414.200012,243.000000,697.000000,0.000007,0.000000,0.000000,-4.000000,0.000000
7,0.000000,-4.000000,0.000000,1024.250000,436.000000,1400.000000,0.375000,798.799988,436.000000,1197.000000,0.000006,0.000000,0.000000,-4.000000,0.000000
8,0.000000,-4.000000,0.000000,691.125000,56.000000,1400.000000,0.250000,454.833344,56.000000,815.000000,0.000008,0.000000,0.000000,-4.000000,0.000000
9,0.000000,-3.625000,0.750000,411.250000,176.000000,860.000000,0.000000,411.250000,176.000000,860.000000,0.000008,0.000000,0.000000,-3.625000,1.060660
10,0.000000,-4.000000,0.000000,770.750000,45.000000,1400.000000,0.375000,393.200012,45.000000,603.000000,0.000008,0.000000,0.000000,-4.000000,0.000000


### Kiểm tra Model sau khi Training
<a name="Inference"></a>

In [ ]:
# Test model without GRPO training (base SFT model)
test_question = "Tốc độ tối đa cho phép trong khu vực đông dân cư là bao nhiêu?"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": test_question},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    tokenize = False,
)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.7,
    top_p = 0.9,
    max_tokens = 512,
)

print("=== Model TRƯỚC khi GRPO training ===")
output_before = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

print(f"Question: {test_question}")
print(f"Answer: {output_before}")
print("="*50)

### Lưu và test model đã được GRPO training

In [ ]:
# Save the GRPO trained LoRA
model.save_lora("grpo_vietnamese_legal_lora")
print("Saved GRPO LoRA to: grpo_vietnamese_legal_lora/")

# Test with GRPO trained model
print("=== Model SAU khi GRPO training ===")
output_after = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_vietnamese_legal_lora"),
)[0].outputs[0].text

print(f"Question: {test_question}")
print(f"Answer: {output_after}")
print("="*50)

# Compare responses
print("=== So sánh ===")
print("Trước GRPO:", len(output_before), "characters")
print("Sau GRPO:", len(output_after), "characters")

### Test với nhiều câu hỏi khác nhau

In [ ]:
# Test multiple questions
test_questions = [
    "Mức phạt vi phạm tốc độ trong khu vực đông dân cư là bao nhiêu?",
    "Khi nào bị tước bằng lái xe?",
    "Quy định về nồng độ cồn khi lái xe là gì?",
]

lora_request = model.load_lora("grpo_vietnamese_legal_lora")

for i, question in enumerate(test_questions, 1):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question},
    ]
    
    text = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
    
    output = model.fast_generate(
        [text],
        sampling_params = sampling_params,
        lora_request = lora_request,
    )[0].outputs[0].text
    
    print(f"\n{i}. Question: {question}")
    print(f"   Answer: {output[:200]}{'...' if len(output) > 200 else ''}")
    print("-" * 60)

### Kiểm tra LoRA đã được training

In [ ]:
from safetensors import safe_open

# Verify LoRA weights are non-zero (indicating successful training)
try:
    tensors = {}
    with safe_open("grpo_vietnamese_legal_lora/adapter_model.safetensors", framework = "pt") as f:
        print("LoRA adapter verification:")
        for key in list(f.keys())[:5]:  # Check first 5 tensors
            tensor = f.get_tensor(key)
            n_zeros = (tensor == 0).sum() / tensor.numel()
            n_nonzeros = 1 - n_zeros.item()
            print(f"  {key}: {n_nonzeros:.2%} non-zero weights")
            assert(n_zeros.item() != 1.0), f"Tensor {key} is all zeros!"
    print("✓ LoRA adapters successfully trained (contain non-zero weights)")
except Exception as e:
    print(f"Error checking LoRA: {e}")
    print("This might be normal if training just started.")

Verify LoRA is actually trained!

In [ ]:
from safetensors import safe_open

tensors = {}
with safe_open("grpo_saved_lora/adapter_model.safetensors", framework = "pt") as f:
    # Verify both A and B are non zero
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

Now we load the LoRA and test:

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

### Lưu Model cuối cùng
<a name="Save"></a>

Lưu model dưới nhiều định dạng khác nhau để sử dụng trong production:

In [ ]:
# Lưu model dưới các định dạng khác nhau

# 1. Lưu LoRA adapters
print("Lưu LoRA adapters...")
model.save_pretrained("vietnamese_legal_grpo_lora")
tokenizer.save_pretrained("vietnamese_legal_grpo_lora") 
print("✓ Saved LoRA to: vietnamese_legal_grpo_lora/")

# 2. Lưu merged model 16bit (nếu cần)
if False: # Bật nếu muốn lưu merged model
    print("Lưu merged model 16bit...")
    model.save_pretrained_merged("vietnamese_legal_grpo_merged_16bit", tokenizer, save_method = "merged_16bit")
    print("✓ Saved merged 16bit to: vietnamese_legal_grpo_merged_16bit/")

# 3. Upload to Hugging Face (cần token)
if False: # Bật và cung cấp token nếu muốn upload
    token = ""  # Điền token từ https://huggingface.co/settings/tokens
    model.push_to_hub("your-username/vietnamese-legal-grpo-lora", token=token)
    tokenizer.push_to_hub("your-username/vietnamese-legal-grpo-lora", token=token)

print("\n=== Hoàn thành training Vietnamese Legal GRPO LoRA ===")
print("Files saved:")
print("- grpo_vietnamese_legal_lora/ (LoRA from training)")  
print("- vietnamese_legal_grpo_lora/ (Final LoRA adapters)")
print("- outputs/ (Training outputs and logs)")

### Xuất GGUF cho llama.cpp (tùy chọn)

Nếu muốn sử dụng với llama.cpp hoặc Ollama:

In [ ]:
# Xuất GGUF (bật nếu cần)
if False:
    # Q4_K_M - Khuyến nghị cho sử dụng thực tế
    print("Xuất GGUF Q4_K_M...")
    model.save_pretrained_gguf("vietnamese_legal_grpo", tokenizer, quantization_method = "q4_k_m")
    print("✓ Saved to vietnamese_legal_grpo-unsloth-Q4_K_M.gguf")

    # Q8_0 - Chất lượng cao hơn
    print("Xuất GGUF Q8_0...")  
    model.save_pretrained_gguf("vietnamese_legal_grpo", tokenizer, quantization_method = "q8_0")
    print("✓ Saved to vietnamese_legal_grpo-unsloth-Q8_0.gguf")

    # Upload to HuggingFace
    # model.push_to_hub_gguf("your-username/vietnamese-legal-grpo", tokenizer, 
    #                       quantization_method = ["q4_k_m", "q8_0"], token = "")

print("GGUF export skipped. Enable above if needed.")